# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file = "../WeatherPy/output_data/cities.csv"
df = pd.read_csv(file)
df.head()

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,latitude,longitude,Max Temp (fahrenheit),Wind Speed
0,0,Blairmore,1,CA,1579564413,52,49.60,-114.44,42.80,5.37
1,1,Bluff,100,NZ,1579564207,68,-46.60,168.33,65.75,5.77
2,2,Chapada dos Guimaraes,20,BR,1579564414,74,-15.46,-55.75,84.20,3.36
3,3,Vao,87,NC,1579564310,59,-22.67,167.48,76.71,5.97
4,4,Tsumeb,38,NaN,1579564414,29,-19.23,17.72,74.91,10.40


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
locations = df[["latitude", "longitude"]].astype(float)
humidity = df["Humidity"].astype(float)

In [4]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=df["Humidity"].max(),
                                 point_radius = 3)

fig.add_layer(heat_layer)

fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
max_temp_ideal = df.loc[df["Max Temp (fahrenheit)"] > 70 ]
max_temp_ideal = max_temp_ideal.loc[max_temp_ideal["Max Temp (fahrenheit)"] < 80]
ideal_wind = max_temp_ideal.loc[max_temp_ideal["Wind Speed"] < 10]
no_clouds = ideal_wind.loc[ideal_wind["Cloudiness"] == 0]
hotel_df = no_clouds
hotel_df

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,latitude,longitude,Max Temp (fahrenheit),Wind Speed
42,42,Upington,0,ZA,1579564419,22,-28.45,21.26,75.20,6.93
99,99,Tra Vinh,0,VN,1579564426,76,9.93,106.35,76.66,7.07
168,168,Windhoek,0,NaN,1579564435,27,-22.56,17.08,71.60,4.70
189,189,Yenagoa,0,NG,1579564438,74,4.92,6.26,77.34,2.73
535,535,Pathein,0,MM,1579564487,74,16.78,94.73,73.00,7.63


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 20000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df

/Users/adambilski/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,latitude,longitude,Max Temp (fahrenheit),Wind Speed,Hotel Name
42,42,Upington,0,ZA,1579564419,22,-28.45,21.26,75.20,6.93,
99,99,Tra Vinh,0,VN,1579564426,76,9.93,106.35,76.66,7.07,
168,168,Windhoek,0,NaN,1579564435,27,-22.56,17.08,71.60,4.70,
189,189,Yenagoa,0,NG,1579564438,74,4.92,6.26,77.34,2.73,
535,535,Pathein,0,MM,1579564487,74,16.78,94.73,73.00,7.63,


In [7]:
# params dictionary to update each iteration
params = {
    "radius": 20000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params).json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

/Users/adambilski/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [8]:
hotel_df

,Unnamed: 0,City Name,Cloudiness,Country,Date,Humidity,latitude,longitude,Max Temp (fahrenheit),Wind Speed,Hotel Name
42,42,Upington,0,ZA,1579564419,22,-28.45,21.26,75.20,6.93,Protea Hotel by Marriott Upington
99,99,Tra Vinh,0,VN,1579564426,76,9.93,106.35,76.66,7.07,Coco Riverside Lodge
168,168,Windhoek,0,NaN,1579564435,27,-22.56,17.08,71.60,4.70,Hilton Windhoek
189,189,Yenagoa,0,NG,1579564438,74,4.92,6.26,77.34,2.73,"Ebiis Hotel Etegwe, Yenagoa"
535,535,Pathein,0,MM,1579564487,74,16.78,94.73,73.00,7.63,The First Hotel


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City Name}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["latitude", "longitude"]]

In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.symbol_layer(locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=6,
    info_box_content=hotel_info)
fig.add_layer(marker_layer)
fig
# Display Map


Figure(layout=FigureLayout(height='420px'))